--- Day 9: Rope Bridge ---
This rope bridge creaks as you walk along it. You aren't sure how old it is, or whether it can even support your weight.

It seems to support the Elves just fine, though. The bridge spans a gorge which was carved out by the massive river far below you.

You step carefully; as you do, the ropes stretch and twist. You decide to distract yourself by modeling rope physics; maybe you can even figure out where not to step.

Consider a rope with a knot at each end; these knots mark the head and the tail of the rope. If the head moves far enough away from the tail, the tail is pulled toward the head.

Due to nebulous reasoning involving Planck lengths, you should be able to model the positions of the knots on a two-dimensional grid. Then, by following a hypothetical series of motions (your puzzle input) for the head, you can determine how the tail will move.

Due to the aforementioned Planck lengths, the rope must be quite short; in fact, the head (H) and tail (T) must always be touching (diagonally adjacent and even overlapping both count as touching):


![hello](1.png)


If the head is ever two steps directly up, down, left, or right from the tail, the tail must also move one step in that direction so it remains close enough:


![hello](2.png)



Otherwise, if the head and tail aren't touching and aren't in the same row or column, the tail always moves one step diagonally to keep up:

![hello](3.png)




You just need to work out where the tail goes as the head follows a series of motions. Assume the head and the tail both start at the same position, overlapping.

For example:

![hello](4.png)



This series of motions moves the head right four steps, then up four steps, then left three steps, then down one step, and so on. After each step, you'll need to update the position of the tail if the step means the head is no longer adjacent to the tail. Visually, these motions occur as follows (s marks the starting position as a reference point):

![hello](7.png)
![hello](8.png)
![hello](9.png)
![hello](10.png)
![hello](11.png)
![hello](12.png)



After simulating the rope, you can count up all of the positions the tail visited at least once. In this diagram, s again marks the starting position (which the tail also visited) and # marks other positions the tail visited:

![hello](13.png)

So, there are 13 positions the tail visited at least once.

Simulate your complete hypothetical series of motions. How many positions does the tail of the rope visit at least once?

In [50]:
from typing import Tuple

def distance(head: Tuple[int, int], tail: Tuple[int, int]):
    """max distance between head and tail of bridge
    """
    return max(abs(head[0]-tail[0]), abs(head[1]-tail[1]))

def add(head: Tuple[int, int], direction: Tuple[int, int]):
    """Add elements of two tuples
    """
    return (head[0]+direction[0], head[1]+direction[1])

def move_head(head: Tuple[int, int], tail: Tuple[int, int]):
    """Logic applied when head is in a same row as tail
    
    The tail is moved directly towared head
    
    TODO: Try and make this more DRY, don't like the if statements
    """
    assert head[0]==tail[0] or head[1]==tail[1]
    if head[0]>tail[0]:
        # Right
        return (tail[0]+1, tail[1])
    elif head[0]<tail[0]:
        # Left
        return (tail[0]-1, tail[1])
    elif head[1]>tail[1]:
        # Up
        return (tail[0], tail[1]+1)
    elif head[1]<tail[1]:
        # Down
        return (tail[0], tail[1]-1)
    else:
        raise ValueError(f"Check conditions, Input: {head} {tail}")

def move_diagonally(head: Tuple[int, int], tail: Tuple[int, int]):
    """Logic applied when head is not in a same row as tail
    
    The tail is moved diagonally to be adjacent to head
    
    TODO: Try and make this more DRY, don't like the if statements
    """
    if head[0]>tail[0] and head[1]>tail[1]:
        # First quadrant
        return (tail[0]+1, tail[1]+1)
    elif head[0]<tail[0] and head[1]>tail[1]:
        # Second quadrant
        return (tail[0]-1, tail[1]+1)
    elif head[0]<tail[0] and head[1]<tail[1]:
        # Third quadrant
        return (tail[0]-1, tail[1]-1)
    elif head[0]>tail[0] and head[1]<tail[1]:
        # Forth quadrant
        return (tail[0]+1, tail[1]-1)
    else:
        raise ValueError("x or y in same row")

def apply_instructions(instructions):
    head = (0, 0)
    tail = (0, 0)
    name_to_direction = {
        "U": (0, 1),
        "D": (0, -1),
        "R": (1, 0),
        "L": (-1, 0),
    }
    tails_visited = set([tail])
    for direction_name, move_distance in instructions:
        direction = name_to_direction[direction_name]
        for move_i in range(move_distance):
            head = add(head, direction)
            if distance(head, tail) > 1:
                if head[0]==tail[0] or head[1]==tail[1]:
                    tail = move_head(head, tail)
                else:
                    tail = move_diagonally(head, tail)
            tails_visited.add(tail)
    return len(tails_visited)

def parse_input(instruction_lines):
    parsed_lines = []
    for line in instruction_lines:
        direction_name, distance_string = line.strip().split()
        parsed_lines.append((direction_name, int(distance_string)))
    return parsed_lines

def simulate_rope_physics():
    text_lines = open("input.txt").readlines()
    parsed_lines = parse_input(text_lines)
    print(apply_instructions(parsed_lines))

simulate_rope_physics()

6087


--- Part Two ---
A rope snaps! Suddenly, the river is getting a lot closer than you remember. The bridge is still there, but some of the ropes that broke are now whipping toward you as you fall through the air!

The ropes are moving too quickly to grab; you only have a few seconds to choose how to arch your body to avoid being hit. Fortunately, your simulation can be extended to support longer ropes.

Rather than two knots, you now must simulate a rope consisting of ten knots. One knot is still the head of the rope and moves according to the series of motions. Each knot further down the rope follows the knot in front of it using the same rules as before.

Using the same series of motions as the above example, but with the knots marked H, 1, 2, ..., 9, the motions now occur as follows:

== Initial State ==
```
......
......
......
......
H.....  (H covers 1, 2, 3, 4, 5, 6, 7, 8, 9, s)
```
== R 4 ==
```
......
......
......
......
1H....  (1 covers 2, 3, 4, 5, 6, 7, 8, 9, s)
```
```
......
......
......
......
21H...  (2 covers 3, 4, 5, 6, 7, 8, 9, s)
```
```
......
......
......
......
321H..  (3 covers 4, 5, 6, 7, 8, 9, s)
```
```
......
......
......
......
4321H.  (4 covers 5, 6, 7, 8, 9, s)
```
== U 4 ==
```
......
......
......
....H.
4321..  (4 covers 5, 6, 7, 8, 9, s)
```
```
......
......
....H.
.4321.
5.....  (5 covers 6, 7, 8, 9, s)
```
```
......
....H.
....1.
.432..
5.....  (5 covers 6, 7, 8, 9, s)
```
```
....H.
....1.
..432.
.5....
6.....  (6 covers 7, 8, 9, s)
```
== L 3 ==
```
...H..
....1.
..432.
.5....
6.....  (6 covers 7, 8, 9, s)
```
```
..H1..
...2..
..43..
.5....
6.....  (6 covers 7, 8, 9, s)
```
```
.H1...
...2..
..43..
.5....
6.....  (6 covers 7, 8, 9, s)

== D 1 ==
```..1...
.H.2..
..43..
.5....
6.....  (6 covers 7, 8, 9, s)
```
== R 4 ==
```..1...
..H2..
..43..
.5....
6.....  (6 covers 7, 8, 9, s)
```
```..1...
...H..  (H covers 2)
..43..
.5....
6.....  (6 covers 7, 8, 9, s)
```
```......
...1H.  (1 covers 2)
..43..
.5....
6.....  (6 covers 7, 8, 9, s)
```
```......
...21H
..43..
.5....
6.....  (6 covers 7, 8, 9, s)
```
== D 1 ==
```......
...21.
..43.H
.5....
6.....  (6 covers 7, 8, 9, s)
```
== L 5 ==
```......
...21.
..43H.
.5....
6.....  (6 covers 7, 8, 9, s)
```
```......
...21.
..4H..  (H covers 3)
.5....
6.....  (6 covers 7, 8, 9, s)
```
```......
...2..
..H1..  (H covers 4; 1 covers 3)
.5....
6.....  (6 covers 7, 8, 9, s)
```
```
......
...2..
.H13..  (1 covers 4)
.5....
6.....  (6 covers 7, 8, 9, s)
```
```......
......
H123..  (2 covers 4)
.5....
6.....  (6 covers 7, 8, 9, s)
```
== R 2 ==
```......
......
.H23..  (H covers 1; 2 covers 4)
.5....
6.....  (6 covers 7, 8, 9, s)
```
```......
......
.1H3..  (H covers 2, 4)
.5....
6.....  (6 covers 7, 8, 9, s)
```
Now, you need to keep track of the positions the new tail, 9, visits. In this example, the tail never moves, and so it only visits 1 position. However, be careful: more types of motion are possible than before, so you might want to visually compare your simulated rope to the one above.

Here's a larger example:
```R 5
U 8
L 8
D 3
R 17
D 10
L 25
U 20
```

These motions occur as follows (individual steps are not shown):

== Initial State ==

```..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
...........H..............  (H covers 1, 2, 3, 4, 5, 6, 7, 8, 9, s)
..........................
..........................
..........................
..........................
..........................
```
== R 5 ==
```
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
...........54321H.........  (5 covers 6, 7, 8, 9, s)
..........................
..........................
..........................
..........................
..........................
```

== U 8 ==

```
..........................
..........................
..........................
..........................
..........................
..........................
..........................
................H.........
................1.........
................2.........
................3.........
...............54.........
..............6...........
.............7............
............8.............
...........9..............  (9 covers s)
..........................
..........................
..........................
..........................
..........................
```

== L 8 ==

```
..........................
..........................
..........................
..........................
..........................
..........................
..........................
........H1234.............
............5.............
............6.............
............7.............
............8.............
............9.............
..........................
..........................
...........s..............
..........................
..........................
..........................
..........................
..........................
```
== D 3 ==
```
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
.........2345.............
........1...6.............
........H...7.............
............8.............
............9.............
..........................
..........................
...........s..............
..........................
..........................
..........................
..........................
..........................
```
== R 17 ==
```
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
................987654321H
..........................
..........................
..........................
..........................
...........s..............
..........................
..........................
..........................
..........................
..........................
```
== D 10 ==
```
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
...........s.........98765
.........................4
.........................3
.........................2
.........................1
.........................H
```
== L 25 ==
```
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
...........s..............
..........................
..........................
..........................
..........................
H123456789................
```
== U 20 ==
```H.........................
1.........................
2.........................
3.........................
4.........................
5.........................
6.........................
7.........................
8.........................
9.........................
..........................
..........................
..........................
..........................
..........................
...........s..............
..........................
..........................
..........................
..........................
..........................
```

Now, the tail (9) visits 36 positions (including s) at least once:

```..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
#.........................
#.............###.........
#............#...#........
.#..........#.....#.......
..#..........#.....#......
...#........#.......#.....
....#......s.........#....
.....#..............#.....
......#............#......
.......#..........#.......
........#........#........
.........########.........
```

Simulate your complete series of motions on a larger rope with ten knots. How many positions does the tail of the rope visit at least once?

In [55]:

def subtract(head: Tuple[int, int], tail: Tuple[int, int]):
    return (head[0]-tail[0], head[1]-tail[1])

def negate(head: Tuple[int, int]):
    return (-head[0], -head[1])


def apply_long_rope_instructions(instructions):
    rope = [(0,0) for i in range(10)]
    name_to_direction = {
        "U": (0, 1),
        "D": (0, -1),
        "R": (1, 0),
        "L": (-1, 0),
    }
    tails_visited = set([(0,0)])
    for direction_name, move_distance in instructions:
        for move_i in range(move_distance):
            direction = name_to_direction[direction_name]
            #Move first knot always
            rope[0] = add(rope[0], direction)
            for following_knot_i in range(1, len(rope)):
                leading_knot = rope[following_knot_i-1]
                following_knot = rope[following_knot_i]
                if distance(leading_knot, following_knot) > 1:
                    if leading_knot[0]==following_knot[0] or leading_knot[1]==following_knot[1]:
                        rope[following_knot_i] = move_head(leading_knot, following_knot)
                    else:
                        rope[following_knot_i] = move_diagonally(head=leading_knot, tail=following_knot)           
            tails_visited.add(rope[-1])
    return len(tails_visited)

def simulate_long_rope_physics():
    text_lines = open("input.txt").readlines()
    parsed_lines = parse_input(text_lines)
    print(apply_long_rope_instructions(parsed_lines))

simulate_long_rope_physics()

2493
